In [4]:
config = {
    'seed': 2025,
    'batch_size': 128,
    'max_length': 40,
    'num_epochs': 400,
    'learning_rate': 5e-4,
    'weight_decay': 1e-2,
    'gradient_clip_norm': 1.0,
    'early_stop_patience': 30,
    'model_path': '../scoures/Port-T5',
    'genome_embedding_size': 84,
    'hidden_size': 256,
    'lstm_hidden_layers': 2
}

In [5]:
import pandas as pd

train_df = pd.read_csv("../../data/EC_X_val_40.csv")

In [6]:
from transformers import DataCollatorWithPadding, T5Tokenizer, T5EncoderModel
from mic_model import PrecomputedEmbeddingDataset
import torch

device = 'cuda'

tokenizer = T5Tokenizer.from_pretrained("../../scoures/Port-T5", trust_remote_code=True)
embedding_model = T5EncoderModel.from_pretrained("../../scoures/Port-T5").to(device)

t5_embedding_size = embedding_model.config.hidden_size
train_dataset = PrecomputedEmbeddingDataset(train_df, tokenizer, embedding_model, 0, config['max_length'],
                                            config['batch_size'])

Some weights of the model checkpoint at ../../scoures/Port-T5 were not used when initializing T5EncoderModel: ['decoder.block.16.layer.1.EncDecAttention.v.weight', 'decoder.block.0.layer.2.DenseReluDense.wo.weight', 'decoder.block.18.layer.2.DenseReluDense.wi.weight', 'decoder.block.17.layer.1.EncDecAttention.v.weight', 'decoder.block.13.layer.0.layer_norm.weight', 'decoder.block.12.layer.0.SelfAttention.q.weight', 'decoder.block.21.layer.1.EncDecAttention.q.weight', 'decoder.block.3.layer.0.SelfAttention.o.weight', 'decoder.block.8.layer.1.EncDecAttention.q.weight', 'decoder.block.23.layer.1.EncDecAttention.v.weight', 'decoder.block.19.layer.0.SelfAttention.v.weight', 'decoder.block.5.layer.1.EncDecAttention.k.weight', 'decoder.block.2.layer.2.DenseReluDense.wo.weight', 'decoder.block.8.layer.0.SelfAttention.v.weight', 'decoder.block.15.layer.0.SelfAttention.o.weight', 'decoder.embed_tokens.weight', 'decoder.block.4.layer.1.layer_norm.weight', 'decoder.block.6.layer.2.DenseReluDense.w

Finalizado. Forma del tensor de embeddings: torch.Size([611, 42, 1024])


In [7]:
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=config['batch_size'])


In [10]:
next(iter(dataloader))["embedding"]

torch.Size([128, 42, 1024])